<a href="https://colab.research.google.com/github/Achrajpachauri/Generative-AI-Project/blob/main/Semi_structured_and_Multi_modal_RAG_w_LLaMA2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install langchain unstructured[all-docs] pydantic lxml

In [ ]:
!sudo apt-get install poppler-utils

In [ ]:
!sudo apt-get update

In [ ]:
!sudo apt-get install libleptonica-dev tesseract-ocr libtesseract-dev python3-pil tesseract-ocr-eng tesseract-ocr-script-latn

In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4

In [ ]:
!pip install unstructured-pytesseract
!pip install tesseract-ocr

In [27]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format

In [28]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

In [ ]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

In [39]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32 # Change this value based on your model and your GPU VRAM pool.
    )

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


In [40]:
prompt = "Write a linear regression in python"
prompt_template=f'''SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.

USER: {prompt}

ASSISTANT:
'''

response=lcpp_llm(prompt=prompt_template, max_tokens=256, temperature=0.5, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)

print(response)


{'id': 'cmpl-25e326a8-edfe-44d5-a164-4455c5dcb484', 'object': 'text_completion', 'created': 1714766651, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': "SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.\n\nUSER: Write a linear regression in python\n\nASSISTANT:\n\nTo write a linear regression in Python, you can use the scikit-learn library. Here is an example of how to do this:\n```\nfrom sklearn.linear_model import LinearRegression\nimport pandas as pd\n\n# Load your dataset into a Pandas DataFrame\ndf = pd.read_csv('your_data.csv')\n\n# Create a linear regression object and fit the data\nreg = LinearRegression().fit(df[['x1', 'x2']], df['y'])\n\n# Print the coefficients\nprint(reg.coef_)\n```\nThis will output the coefficients of the linear regression, which you can use to make predictions on new data. For exam

In [4]:
from typing import Any

from pydantic import BaseModel
from unstructured.partition.pdf import partition_pdf




In [ ]:
path = '/content/'

#/content/llama2.pdf

raw_pdf_element = partition_pdf(filename='/content/llama2.pdf',extract_images_in_pdf=True,infer_table_structure=True,chunking_strategy='by_title', max_characters=4000,
    new_after_n_chars=3800,
    combine_text_under_n_chars=2000,
    image_output_dir_path=path)

In [15]:
#seperate category
category_counts = {}

for element in raw_pdf_element:
    category = str(type(element))
    if category in category_counts:
        category_counts[category] += 1
    else:
        category_counts[category] = 1

unique_category = set(category_counts)

category_counts

{"<class 'unstructured.documents.elements.CompositeElement'>": 31,
 "<class 'unstructured.documents.elements.Table'>": 4}

In [19]:
table_elements = []
text_elements = []

categorized_elements = []
for element in raw_pdf_element:
    if "unstructured.documents.elements.Table" in str(type(element)):
        table_elements.append(str(element))
    elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
        text_elements.append(str(element))

table_elements[0]
#text_elements[0]

'Conversation Detail description Complex reasoning All Full data Detail + Complex Conv + 5% Detail + 10% Complex Conversation No Instruction Tuning 83.1 81.5 (-1.6) 81.0 (-2.1) 76.5 (-6.6) 22.0 (-61.1) 75.3 73.3 (-2.0) 68.4 (-7.1) 59.8 (-16.2) 24.0 (-51.3) 96.5 90.8 (-5.7) 91.5 (-5.0) 84.9 (-12.4) 18.5 (-78.0) 85.1 81.9 (-3.2) 80.5 (-4.4) 73.8 (-11.3) 21.5 (-63.6)'

In [20]:
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [37]:
# Prompt
prompt_text = """You are an assistant tasked with summarizing tables and text. \
Give a concise summary of the table or text. Table or text chunk: {element} """
prompt = ChatPromptTemplate.from_template(prompt_text)

# Summary chain
model = ChatOllama(model=model_path )
summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()

In [38]:
print(summarize_chain)

first={
  element: RunnableLambda(lambda x: x)
} middle=[ChatPromptTemplate(input_variables=['element'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['element'], template='You are an assistant tasked with summarizing tables and text. Give a concise summary of the table or text. Table or text chunk: {element} '))]), ChatOllama(model='/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin')] last=StrOutputParser()


In [36]:
texts = [element for element in text_elements if element != ""]
text_summaries = summarize_chain.batch(texts, {"max_concurrency": 5})

ConnectionError: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/chat (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7e5bfffcad10>: Failed to establish a new connection: [Errno 111] Connection refused'))